## Langchain 

#####  What is LangChain?

LangChain is a framework that helps you build applications using Large Language Models (LLMs) like GPT, Claude, Gemini, etc.

##### Why do we need LangChain?

LLMs cannot do these things by themselves:

+ Remember past messages
+ Search your documents
+ Connect to databases
+ Call APIs
+ Use tools
+ Run multi-step reasoning
+ Build workflows

LangChain provides:

- Memory >> LLM remembers previous conversation
- Retrieval (RAG) >> LLM can fetch data from PDFs, websites, DBs
- Agents >> LLM can think, decide, and call tools
- Chains >> multi-step pipelines
- Embeddings + Vector DB >> store and search knowledge
- Tools >> like Python, search, calculators
- LangGraph >> build complex LLM workflows
- LangSmith >> evaluation and debugging

### LangChain Ecosystem


The LangChain ecosystem is made of 5 main libraries, each with a specific purpose.
Understanding these makes the entire framework easy.

##### langchain-core:
The foundation of the LangChain ecosystem.

It contains the following: 
+ Prompts (ChatPromptTemplate)
+ Runnables / Chains (LCEL pipelines)
+ Tools framework
+ Messages (HumanMessage, AIMessage)
+ Output parsers
+ Schema definitions

##### langchain-community

A big library of community-contributed integrations. Also includes the below ones.

+ Vector stores (FAISS, Pinecone, Chroma…)
+ Document loaders (PDF, Excel, websites)
+ Embedding models
+ Tools (search APIs, calculators)
+ Older chain utilities

##### langchain-openai
The OpenAI integration package >>>>  models, embeddings, chat completions. This keeps LangChain modular. It makes easy to switch model providers and also contains the following

+ ChatOpenAI
+ OpenAIEmbeddings
+ OpenAI client wrappers

This is bridge between LangChain and OpenAI/GPT models.

##### langgraph
A workflow engine for building agents and complex multi-step AI systems. This is where agents officially live now (not inside LangChain). It supports:

+ ReAct agents
+ State machines
+ Loops & conditional workflows
+ Human-in-the-loop
+ Multi-agent systems

The brain + control flow for complex AI reasoning.
LangChain = functions, LangGraph = workflows.

#### langsmith
A platform for debugging, evaluating, and monitoring LLM apps. It provides the following

+ Traces of every step in a chain/agent
+ Dataset evaluation
+ Comparison across models
+ Prompt debugging
+ Performance analytics

The observability & monitoring layer for LangChain applications.
LangSmith automatically logs chain/agent calls if enabled.

In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

#### Invoke the model

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini")
response = llm.invoke("What is LangChain?")
print(response.content)

LangChain is an open-source framework designed for developing applications that use large language models (LLMs). It provides a structured approach to building applications that make use of the capabilities of LLMs, enabling developers to create more sophisticated and feature-rich applications. Launched in 2022, LangChain offers a variety of components and tools that help facilitate tasks such as:

1. **Chains**: Sequences of calls to LLMs or other processing functions, allowing for complex workflows to be built.

2. **Agents**: A mechanism that enables LLMs to interact with their environment, accept user input, and make decisions based on that input.

3. **Data Augmented Generation**: Integrating external data sources into the LLM's responses to improve relevancy and accuracy.

4. **Memory**: Options for maintaining context and state in conversational AI applications, allowing models to remember past interactions.

5. **Integrations**: Compatibility with various data sources, APIs, an

#### Chain: Combine prompt >> LLM into one workflow

A Chain is a sequence of operations connected together to process input and produce output using an LLM.

In [6]:
from langchain_core.prompts import ChatPromptTemplate

In [7]:
prompt = ChatPromptTemplate.from_template("Explain {topic} in one line.")
llm = ChatOpenAI(model="gpt-4o-mini")

chain = prompt | llm

print(chain.invoke({"topic": "Agentic AI"}).content)

Agentic AI refers to artificial intelligence systems that can make autonomous decisions and take actions independently to achieve specific goals.


#### Tool: External function the LLM can call

In [9]:
from langchain_core.tools import tool 

@tool
def add_numbers(x: int, y: int) -> int:
    """Add two integers and return the result."""
    return x + y

result = add_numbers.invoke({"x": 5, "y": 10})
print(result)  # 15


15


#### Agent: LLM that decides which tool to use

In [11]:
import langchain
print(langchain.__version__)

1.1.3


In [39]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

# --- Define Tool ---
@tool
def reverse_text(text: str) -> str:
    """Reverse the input text."""
    return text[::-1]

tools = [reverse_text]

# --- LLM ---
model = ChatOpenAI(model="gpt-4o-mini")

# --- Create Agent (NO prompt needed) ---
agent = create_react_agent(
    model=model,        # Correct argument name
    tools=tools
)

# --- Run it ---
result = agent.invoke({
    "messages": [
        ("user", "Reverse the word LangChain")
    ]
})

print(result)

print(result["messages"][-1].content)

C:\Users\LotusBlue\AppData\Local\Temp\ipykernel_24772\89202948.py:17: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


The reversed word "LangChain" is "niahCgnaL".


#### Memory: Store and recall previous messages.

In [60]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

# LLM
llm = ChatOpenAI(model="gpt-4o-mini")

# Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}")
])

chain = prompt | llm

# Store memory for each session
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# Wrap chain with memory
with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="messages"
)

# Interactions
print(with_memory.invoke(
    {"input": "My name is Keerthi"},
    config={"configurable": {"session_id": "abc"}}
))

print(with_memory.invoke(
    {"input": "What is my name?"},
    config={"configurable": {"session_id": "abc"}}
))


content='Hi Keerthi! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 23, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_11f3029f6b', 'id': 'chatcmpl-Cl2wdfSVKLd2hHFCwoeZktvvM6pMb', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b05cb-5203-7132-9b79-552cd85c68f2-0' usage_metadata={'input_tokens': 23, 'output_tokens': 12, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content="I'm sorry, but I don't know your name. I can't access personal information unless you provide it. How can I assist you today?" additiona